In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pytorch-lightning  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 4.3 MB/s 
     |████████████████████████████████| 419 kB 43.4 MB/s 
     |████████████████████████████████| 140 kB 49.5 MB/s 
     |████████████████████████████████| 596 kB 38.9 MB/s 
     |████████████████████████████████| 1.1 MB 28.8 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 44.9 MB/s 
     |████████████████████████████████| 271 kB 52.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install pytorch-forecasting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 4.2 MB/s 
     |████████████████████████████████| 308 kB 28.0 MB/s 
     |████████████████████████████████| 209 kB 47.0 MB/s 
     |████████████████████████████████| 81 kB 9.9 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
     |████████████████████████████████| 112 kB 16.8 MB/s 
     |████████████████████████████████| 146 kB 35.0 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=2e5a4432fcc460bb52f77c331af111ef7c974e71c24f8bad7fae032c63bf1659
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [4]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from pytorch_forecasting import Baseline

import joblib
import pickle
import optuna

warnings.filterwarnings('ignore')
%matplotlib inline

### 함수정의

In [2]:
def sliding_windows(data, lookback_length, forecast_length):

    x = []
    y = []
    
    for i in range(lookback_length, len(data) - forecast_length + 1):
        _x = data[(i-lookback_length) : i]
        _y = data[i : (i + forecast_length)]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)


def get_data_loader(X, y, batch_size):

    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    train_dl = DataLoader(train_ds, batch_size = batch_size)

    val_ds = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    val_dl = DataLoader(val_ds, batch_size = batch_size)

    input_size = x_train.shape[-1]

    return train_dl, val_dl, input_size

### Data loading && Preproces

In [3]:
scaler = MinMaxScaler()

data=pd.read_excel('./data_full.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             1858 non-null   datetime64[ns]
 1   Account DOW      1858 non-null   object        
 2   REV OBD          1858 non-null   int64         
 3   OBD NET+FSC_KRW  1858 non-null   float64       
 4   OBD A/R_KRW      1858 non-null   float64       
 5   REV CPN          1858 non-null   int64         
 6   CPN NET+FSC_KRW  1858 non-null   float64       
 7   CPN A/R_KRW      1858 non-null   float64       
 8   REV TKT          1858 non-null   int64         
 9   TKT NET+FSC_KRW  1858 non-null   float64       
 10  TKT A/R_KRW      1858 non-null   float64       
 11  WTI              1858 non-null   float64       
 12  exchanges        1858 non-null   float64       
 13  kospi            1858 non-null   float64       
 14  rates            1858 non-null   float64

In [4]:
res_data = data[data["Date"].isin(pd.date_range('2016-01-04', '2019-11-30'))]
res_data.reset_index(drop=True, inplace=True)
res_data = res_data[['Date', 'REV OBD']]
scale_cols = ['REV OBD']

In [5]:
res_data.shape

(1427, 2)

In [6]:
# 원본 코드 LSTM-pytorch_optuna_05와 다른 부분

# Loockback_period & forecasting_period
max_prediction_length = 20
lookback_length = 100
training_data_max = len(res_data) - max_prediction_length

# 학습용 데이터
data_p = res_data.iloc[:training_data_max, :]
training_data = scaler.fit_transform(data_p[scale_cols])

In [7]:
training_data_max

1407

In [8]:
data_p.shape

(1407, 2)

In [9]:
data_p

,Date,REV OBD
0,2016-01-04,28000
1,2016-01-05,24657
2,2016-01-06,26920
3,2016-01-07,26624
4,2016-01-08,28879
...,...,...
1402,2019-11-06,25486
1403,2019-11-07,25516
1404,2019-11-08,30883
1405,2019-11-09,27640


#### Metric 생성을 위한 oot sample 정의 (원본 코드 LSTM-pytorch_optuna_05와 다른 부분)


In [10]:
# max_prediction_length 만큼의 데이터는 예측 데이터와 비교를 위해 분리
# Training set에 없는 데이터로 구성
# Input과 output의 pair로 정의
x_for_metric = scaler.fit_transform(res_data[training_data_max -lookback_length : training_data_max][scale_cols])
y_for_metric = scaler.fit_transform(res_data[training_data_max:][scale_cols])

In [11]:
x_for_metric.shape

(100, 1)

In [12]:
y_for_metric.shape

(20, 1)

In [13]:
# LSTM은 1 step 뒤의 값만을 예측하므로, forecasting_period를 1로 두고 진행
x, y = sliding_windows(training_data, lookback_length, 1)

### Model 정의

In [14]:
# 원본 코드 LSTM-pytorch_optuna_05와 다른 부분

class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, batch_first = True)
        
        self.fc = nn.Linear(hidden_size  * num_layers, num_classes)
        
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size * self.num_layers)
        out = self.fc(h_out)
        return out

In [15]:
def train(log_interval, model, train_dl, val_dl, optimizer, criterion, epoch):

    best_loss = np.inf
    for epoch in range(epoch):
        train_loss = 0.0
        model.train()
        for data, target in train_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()
                model = model.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target) # mean-squared error for regression
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # validation
        valid_loss = 0.0
        model.eval()
        for data, target in val_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()

            output = model(data)         
            loss = criterion(output, target)
            valid_loss += loss.item()

        if ( epoch % log_interval == 0 ):
            print(f'\n Epoch {epoch} \t Training Loss: {train_loss / len(train_dl)} \t Validation Loss: {valid_loss / len(val_dl)} \n')

        if best_loss > (valid_loss / len(val_dl)):
            print(f'Validation Loss Decreased({best_loss:.6f}--->{(valid_loss / len(val_dl)):.6f}) \t Saving The Model')
            best_loss = (valid_loss / len(val_dl))
            torch.save(model.state_dict(), 'lstm_saved_model.pth')

    return best_loss


def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [16]:
def objective(trial):

    cfg = { 
            'batch_size' : trial.suggest_categorical('batch_size',[32, 64, 128, 256, 512]), # [64, 128, 256],
            'num_epochs' : trial.suggest_int('num_epochs', 5, 50, 5), # 파라미터라고 보기 어려움
            'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 1e-1), #trial.suggest_loguniform('learning_rate', 1e-2, 1e-1), # learning rate을 0.01-0.1까지 로그 uniform 분포로 사용
            'hidden_size': trial.suggest_categorical('hidden_size',[32, 64, 128, 256, 512, 1024]),
            'num_layers': trial.suggest_int('num_layers', 1, 5, 1),       
        }

    torch.manual_seed(42)

    log_interval = 5
    num_classes = 1 # parameter에서 빼서 상수로 설정

    train_dl, val_dl, input_size = get_data_loader(x, y,  cfg['batch_size'])
    
    model = LSTM(
        num_classes = num_classes, 
        input_size = input_size, 
        hidden_size = cfg['hidden_size'], 
        num_layers = cfg['num_layers']
    )
    
    if torch.cuda.is_available():
        model = model.cuda()
        
    optimizer = optim.Adam(model.parameters(), lr=cfg['learning_rate'])
    criterion = torch.nn.MSELoss()
    best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, cfg['num_epochs'])

    print('best loss for the trial = ', best_loss)
    predict_data = []
    # 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
    x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

    for j, i in enumerate(range(max_prediction_length)):

        # feed the last forecast back to the model as an input
        x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
        xt_pred = torch.Tensor(x_pred)

        if torch.cuda.is_available():
            xt_pred = xt_pred.cuda()
        # generate the next forecast
        yt_pred = model(xt_pred)
        # tensor to array
        # x_pred = xt_pred.cpu().detach().numpy()
        y_pred = yt_pred.cpu().detach().numpy()

        # save the forecast
        predict_data.append(y_pred)

    # transform the forecasts back to the original scale
    predict_data = np.array(predict_data).reshape(-1, 1)
    SMAPE = smape(y_for_metric, predict_data)
    
    print(f' \nSMAPE : {SMAPE}')


    return SMAPE

In [17]:
sampler = optuna.samplers.TPESampler()
#   sampler = optuna.samplers.SkoptSampler()

# model.load_state_dict(torch.load('lstm_saved_model.pth'))
    
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(objective, n_trials= 5)
joblib.dump(study, '/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_05.pkl')

[I 2022-06-19 19:08:52,806] A new study created in memory with name: no-name-5cedd8f0-3e81-4e0c-87b1-b3127bead622



 Epoch 0 	 Training Loss: 15.980667213598887 	 Validation Loss: 2.2843921979268393 

Validation Loss Decreased(inf--->2.284392) 	 Saving The Model
Validation Loss Decreased(2.284392--->0.079399) 	 Saving The Model
Validation Loss Decreased(0.079399--->0.025409) 	 Saving The Model
best loss for the trial =  0.025409390528996784


[I 2022-06-19 19:09:10,533] Trial 0 finished with value: 102.43403738110736 and parameters: {'batch_size': 128, 'num_epochs': 5, 'learning_rate': 0.005372846974718744, 'hidden_size': 1024, 'num_layers': 3}. Best is trial 0 with value: 102.43403738110736.


 
SMAPE : 102.43403738110736

 Epoch 0 	 Training Loss: 0.17669273954298761 	 Validation Loss: 0.07971998676657677 

Validation Loss Decreased(inf--->0.079720) 	 Saving The Model
Validation Loss Decreased(0.079720--->0.020465) 	 Saving The Model
Validation Loss Decreased(0.020465--->0.018382) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.024157104806767568 	 Validation Loss: 0.01763868673394124 

Validation Loss Decreased(0.018382--->0.017639) 	 Saving The Model

 Epoch 10 	 Training Loss: 0.022973534961541493 	 Validation Loss: 0.018054944773515064 


 Epoch 15 	 Training Loss: 0.023134491095940273 	 Validation Loss: 0.018070284898082416 


 Epoch 20 	 Training Loss: 0.02313618713782893 	 Validation Loss: 0.018078239013751347 


 Epoch 25 	 Training Loss: 0.023106680148177676 	 Validation Loss: 0.018068363269170124 


 Epoch 30 	 Training Loss: 0.02307067231999503 	 Validation Loss: 0.018052250146865845 


 Epoch 35 	 Training Loss: 0.023034800464908283 	 Validation Loss: 0.01803591

[I 2022-06-19 19:09:20,257] Trial 1 finished with value: 68.99157830090309 and parameters: {'batch_size': 128, 'num_epochs': 50, 'learning_rate': 0.024508823124406696, 'hidden_size': 64, 'num_layers': 4}. Best is trial 1 with value: 68.99157830090309.


best loss for the trial =  0.01763868673394124
 
SMAPE : 68.99157830090309

 Epoch 0 	 Training Loss: 0.06221604235470295 	 Validation Loss: 0.016891437582671642 

Validation Loss Decreased(inf--->0.016891) 	 Saving The Model
Validation Loss Decreased(0.016891--->0.016657) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.022441845387220383 	 Validation Loss: 0.018641767092049122 

Validation Loss Decreased(0.016657--->0.016485) 	 Saving The Model

 Epoch 10 	 Training Loss: 0.02239111326634884 	 Validation Loss: 0.017094753216952085 


 Epoch 15 	 Training Loss: 0.022406283020973205 	 Validation Loss: 0.017064202576875687 


 Epoch 20 	 Training Loss: 0.02237342856824398 	 Validation Loss: 0.017159316688776016 


 Epoch 25 	 Training Loss: 0.022381429374217988 	 Validation Loss: 0.0171490921638906 


 Epoch 30 	 Training Loss: 0.022387510910630227 	 Validation Loss: 0.017138131894171238 



[I 2022-06-19 19:09:25,974] Trial 2 finished with value: 66.48239054557986 and parameters: {'batch_size': 256, 'num_epochs': 35, 'learning_rate': 0.0017735999440525484, 'hidden_size': 256, 'num_layers': 2}. Best is trial 2 with value: 66.48239054557986.


best loss for the trial =  0.016484765335917473
 
SMAPE : 66.48239054557986

 Epoch 0 	 Training Loss: 0.14486036449670792 	 Validation Loss: 0.06612972915172577 

Validation Loss Decreased(inf--->0.066130) 	 Saving The Model
Validation Loss Decreased(0.066130--->0.032833) 	 Saving The Model
Validation Loss Decreased(0.032833--->0.023709) 	 Saving The Model
Validation Loss Decreased(0.023709--->0.021412) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.022633306061228115 	 Validation Loss: 0.019172124564647675 

Validation Loss Decreased(0.021412--->0.019172) 	 Saving The Model

 Epoch 10 	 Training Loss: 0.022224045048157375 	 Validation Loss: 0.01911080814898014 

Validation Loss Decreased(0.019172--->0.019111) 	 Saving The Model
Validation Loss Decreased(0.019111--->0.018973) 	 Saving The Model

 Epoch 15 	 Training Loss: 0.022216464082400005 	 Validation Loss: 0.019112059846520424 

Validation Loss Decreased(0.018973--->0.018935) 	 Saving The Model
Validation Loss Decreased(0.018935

[I 2022-06-19 19:09:27,866] Trial 3 finished with value: 66.66892622649793 and parameters: {'batch_size': 512, 'num_epochs': 25, 'learning_rate': 0.0020589421256473808, 'hidden_size': 128, 'num_layers': 2}. Best is trial 2 with value: 66.48239054557986.


best loss for the trial =  0.018893690779805183
 
SMAPE : 66.66892622649793

 Epoch 0 	 Training Loss: 435.16487167911095 	 Validation Loss: 21.282385932074654 

Validation Loss Decreased(inf--->21.282386) 	 Saving The Model
Validation Loss Decreased(21.282386--->2.837904) 	 Saving The Model
Validation Loss Decreased(2.837904--->0.077366) 	 Saving The Model
Validation Loss Decreased(0.077366--->0.025989) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.04844417504853371 	 Validation Loss: 0.040588608011603355 

Validation Loss Decreased(0.025989--->0.023736) 	 Saving The Model
Validation Loss Decreased(0.023736--->0.019687) 	 Saving The Model

 Epoch 10 	 Training Loss: 0.05179679977961562 	 Validation Loss: 0.025877195927831862 

Validation Loss Decreased(0.019687--->0.019014) 	 Saving The Model

 Epoch 15 	 Training Loss: 0.08734113147312944 	 Validation Loss: 0.10641986048883861 



In [22]:
study = joblib.load('/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_05.pkl') 
df = study.trials_dataframe() 
df.sort_values('value')

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_hidden_size,params_learning_rate,params_num_epochs,params_num_layers,state
3,3,61.683910,2022-06-18 23:59:06.503369,2022-06-18 23:59:10.220675,0 days 00:00:03.717306,64,32,0.001878,30,1,COMPLETE
4,4,62.946093,2022-06-18 23:59:10.224759,2022-06-18 23:59:14.325827,0 days 00:00:04.101068,64,64,0.028660,30,1,COMPLETE
0,0,63.670804,2022-06-18 23:58:24.279572,2022-06-18 23:58:31.366290,0 days 00:00:07.086718,64,64,0.002045,25,3,COMPLETE
1,1,69.241567,2022-06-18 23:58:31.368762,2022-06-18 23:58:38.950067,0 days 00:00:07.581305,128,32,0.017326,30,5,COMPLETE
2,2,200.000000,2022-06-18 23:58:38.952322,2022-06-18 23:59:06.501205,0 days 00:00:27.548883,32,256,0.065448,25,4,COMPLETE


In [23]:
study.best_params

{'batch_size': 64,
 'hidden_size': 32,
 'learning_rate': 0.0018782118513314694,
 'num_epochs': 30,
 'num_layers': 1}

In [24]:
optuna.visualization.plot_param_importances(study)

In [25]:
optuna.visualization.plot_optimization_history(study)

In [26]:
torch.manual_seed(42)

log_interval = 5
num_classes = 1 # parameter에서 빼서 상수로 설정

train_dl, val_dl, input_size = get_data_loader(x, y,  study.best_params['batch_size'])

model = LSTM(
    num_classes = num_classes, 
    input_size = input_size, 
    hidden_size = study.best_params['hidden_size'], 
    num_layers = study.best_params['num_layers']
)

if torch.cuda.is_available():
    model = model.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=study.best_params['learning_rate'])
criterion = torch.nn.MSELoss()
best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, study.best_params['num_epochs'])

print('best loss for the trial = ', best_loss)
predict_data = []
# 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

for j, i in enumerate(range(max_prediction_length)):

    # feed the last forecast back to the model as an input
    x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
    xt_pred = torch.Tensor(x_pred)

    if torch.cuda.is_available():
        xt_pred = xt_pred.cuda()
    # generate the next forecast
    yt_pred = model(xt_pred)
    # tensor to array
    # x_pred = xt_pred.cpu().detach().numpy()
    y_pred = yt_pred.cpu().detach().numpy()

    # save the forecast
    predict_data.append(y_pred)

# transform the forecasts back to the original scale
predict_data = np.array(predict_data).reshape(-1, 1)
SMAPE = smape(y_for_metric, predict_data)

print(f' \nSMAPE : {SMAPE}')


 Epoch 0 	 Training Loss: 0.09048183481482898 	 Validation Loss: 0.021324258483946323 

Validation Loss Decreased(inf--->0.021324) 	 Saving The Model
Validation Loss Decreased(0.021324--->0.018006) 	 Saving The Model
Validation Loss Decreased(0.018006--->0.017908) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.022353263900560492 	 Validation Loss: 0.018062368035316467 


 Epoch 10 	 Training Loss: 0.0224425766397925 	 Validation Loss: 0.018069243431091307 


 Epoch 15 	 Training Loss: 0.02243964176844148 	 Validation Loss: 0.018061177618801594 


 Epoch 20 	 Training Loss: 0.02242254964349901 	 Validation Loss: 0.018048238195478915 


 Epoch 25 	 Training Loss: 0.02240299225291785 	 Validation Loss: 0.018035004660487176 

best loss for the trial =  0.017907843738794327
 
SMAPE : 61.68390990969686


In [29]:
# actual_data는 max_prediction_length에 해당하는 20일의 기간 : 실제 예측하려는 기간 (2019년 12월1일 ~ 2019년 12월 20일)
actual_data = data[data["Date"].isin(pd.date_range('2019-12-01', '2019-12-20'))]
actual_data['time_index'] = np.arange(len(actual_data))
actual_data['market'] = 'OBD'
actual_data.head()

,Date,Account DOW,REV OBD,OBD NET+FSC_KRW,OBD A/R_KRW,REV CPN,CPN NET+FSC_KRW,CPN A/R_KRW,REV TKT,TKT NET+FSC_KRW,TKT A/R_KRW,WTI,exchanges,kospi,rates,stock_a,stock_k,stock_kkj,time_index,market
1430,2019-12-01,SUN,28090,1.093871e+10,389416.4866,10669,3.644840e+09,341629.0611,5973,3.888420e+09,650999.5691,55.17,1180.0,2087.96,1.39,25391,16048,47426,0,OBD
1431,2019-12-02,MON,27057,9.887493e+09,365431.9919,40638,1.622895e+10,399354.0099,21846,1.747528e+10,799930.3397,55.96,1185.0,2091.92,1.43,24770,16048,47377,1,OBD
1432,2019-12-03,TUE,22931,7.988729e+09,348381.2026,37530,1.512760e+10,403080.2320,20447,1.633829e+10,799055.6766,56.10,1190.0,2084.07,1.49,24818,15854,46009,2,OBD
1433,2019-12-04,WED,23651,8.167825e+09,345347.9761,36150,1.466185e+10,405583.5527,19725,1.582003e+10,802029.2492,58.43,1193.0,2068.89,1.41,24436,15724,45618,3,OBD
1434,2019-12-05,THU,25570,9.347876e+09,365579.8221,37518,1.617110e+10,431022.5730,20295,1.755050e+10,864769.6796,58.43,1191.0,2060.74,1.42,24245,15919,46107,4,OBD


In [55]:
# 실제 예측하려는 target prediction 구간 : 20일
actuals = actual_data['REV OBD']
actuals

1430    28090
1431    27057
1432    22931
1433    23651
1434    25570
1435    29559
1436    26113
1437    28809
1438    27316
1439    23017
1440    25317
1441    25544
1442    29799
1443    27079
1444    29413
1445    29138
1446    24957
1447    28161
1448    27917
1449    32165
Name: REV OBD, dtype: int64

In [56]:
actuals = torch.tensor(actual_data['REV OBD'].values)
print(actuals)
print(actuals.shape)

tensor([28090, 27057, 22931, 23651, 25570, 29559, 26113, 28809, 27316, 23017,
        25317, 25544, 29799, 27079, 29413, 29138, 24957, 28161, 27917, 32165])
torch.Size([20])


In [57]:
baseline_predictions =  torch.tensor([data.loc[data.Date == '2019-11-30']['REV OBD'].values[0]] * len(actuals)).to(torch.float32)

mae_with_baseline = (actuals - baseline_predictions).abs().mean().item()
smape_with_baseline = (2 * (baseline_predictions - actuals).abs() / (baseline_predictions.abs() + actuals.abs() + 1e-8)).mean()

print(f'mae_with_baseline : {mae_with_baseline}')
print(f'smape_with_baseline : {smape_with_baseline}')

mae_with_baseline : 2287.35009765625
smape_with_baseline : 0.08517351746559143


In [63]:
predicts = scaler.inverse_transform(predict_data)
print(predicts.shape)
print(predicts)

(20, 1)
[[25832.44 ]
 [25626.494]
 [25634.81 ]
 [25668.055]
 [25935.832]
 [26046.277]
 [26251.129]
 [26332.691]
 [25992.01 ]
 [25931.81 ]
 [25909.61 ]
 [26140.588]
 [26147.004]
 [26231.732]
 [26279.799]
 [25921.818]
 [25768.852]
 [25659.232]
 [25864.012]
 [25833.652]]


In [64]:
predicts = torch.tensor(predicts)
predicts.shape

torch.Size([20, 1])

In [65]:
actuals = actuals.reshape(-1,1)
actuals.shape

torch.Size([20, 1])

In [66]:
mae_with_test = (actuals - predicts).abs().mean().item()
smape_with_test = smape(actuals, predicts)

print(f'mae_with_test : {mae_with_test}')
print(f'smape_with_test : {smape_with_test}')

TypeError: ignored

In [69]:
mae_with_test = (actuals - predicts).abs().mean().item()
mae_with_test 

2143.826171875

In [ ]:
y_train.shape

np.expand_dims(pred, axis=1).shape

predicted_for_plot = sc.inverse_transform(np.concatenate((y_train, np.expand_dims(pred, axis=1))))
actual_for_plot = sc.inverse_transform(np.concatenate((y_train, y_test)))

predicted_for_plot.shape

actual_for_plot.shape

plt.plot(predicted_for_plot[300:])
plt.plot(actual_for_plot[300:])
plt.suptitle('Timeseries Prediction')
plt.axvline(x=y_train.shape[0]-300, c='r', linestyle='--')
plt.show()

NameError: ignored